<a href="https://colab.research.google.com/github/pragatipatel26/mini_assignment1/blob/main/Major_assignment1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
from google.colab import files
uploaded = files.upload()


import os
os.listdir()



Saving Data Dictionary.xlsx to Data Dictionary.xlsx
Saving Test.csv to Test.csv
Saving Train.csv to Train.csv
Saving Valid.csv to Valid.csv


['.config',
 'Test.csv',
 'drive',
 'Data Dictionary.xlsx',
 'Valid.csv',
 'Train.csv',
 'sample_data']

In [7]:
# Load the datasets
import pandas as pd

train_df = pd.read_csv('Train.csv')
test_df = pd.read_csv('Test.csv')
valid_df = pd.read_csv('Valid.csv')
data_dict=pd.read_excel('Data Dictionary.xlsx')

<ipython-input-7-797525647>:4: DtypeWarning: Columns (13,39,40,41) have mixed types. Specify dtype option on import or set low_memory=False.
  train_df = pd.read_csv('Train.csv')
/usr/local/lib/python3.11/dist-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


In [8]:
print("Train Shape:", train_df.shape)
print("Test Shape:", test_df.shape)
print("Valid Shape:", valid_df.shape)

Train Shape: (401125, 53)
Test Shape: (12457, 52)
Valid Shape: (11573, 52)


In [146]:
train_df.info()
# Count missing values
missing_values = train_df.isnull().sum()
missing_values = missing_values[missing_values > 0].sort_values(ascending=False)
print("Missing values:\n", missing_values)
# Quick summary stats to help find outliers
train_df.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 401125 entries, 0 to 401124
Data columns (total 53 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   SalesID                   401125 non-null  int64  
 1   SalePrice                 401125 non-null  int64  
 2   MachineID                 401125 non-null  int64  
 3   ModelID                   401125 non-null  int64  
 4   datasource                401125 non-null  int64  
 5   auctioneerID              380989 non-null  float64
 6   YearMade                  401125 non-null  int64  
 7   MachineHoursCurrentMeter  142765 non-null  float64
 8   UsageBand                 69639 non-null   object 
 9   saledate                  401125 non-null  object 
 10  fiModelDesc               401125 non-null  object 
 11  fiBaseModel               401125 non-null  object 
 12  fiSecondaryDesc           263934 non-null  object 
 13  fiModelSeries             56908 non-null   o

,SalesID,SalePrice,MachineID,ModelID,datasource,auctioneerID,YearMade,MachineHoursCurrentMeter
count,4.011250e+05,401125.000000,4.011250e+05,401125.000000,401125.000000,380989.000000,401125.000000,1.427650e+05
mean,1.919713e+06,31099.712848,1.217903e+06,6889.702980,134.665810,6.556040,1899.156901,3.457955e+03
std,9.090215e+05,23036.898502,4.409920e+05,6221.777842,8.962237,16.976779,291.797469,2.759026e+04
min,1.139246e+06,4750.000000,0.000000e+00,28.000000,121.000000,0.000000,1000.000000,0.000000e+00
25%,1.418371e+06,14500.000000,1.088697e+06,3259.000000,132.000000,1.000000,1985.000000,0.000000e+00
50%,1.639422e+06,24000.000000,1.279490e+06,4604.000000,132.000000,2.000000,1995.000000,0.000000e+00
75%,2.242707e+06,40000.000000,1.468067e+06,8724.000000,136.000000,4.000000,2000.000000,3.025000e+03
max,6.333342e+06,142000.000000,2.486330e+06,37198.000000,172.000000,99.000000,2013.000000,2.483300e+06


In [147]:
# Handle missing values
missing_ratio = train_df.isnull().mean()
cols_to_drop = missing_ratio[missing_ratio > 0.3].index.tolist()

print(f"\nDropping {len(cols_to_drop)} columns due to >30% missing:")
print(cols_to_drop)

train = train_df.drop(columns=[col for col in cols_to_drop if col in train_df.columns])
valid = valid_df.drop(columns=[col for col in cols_to_drop if col in valid_df.columns])
test  = test_df.drop(columns=[col for col in cols_to_drop if col in test_df.columns])

print("\nShapes after safe column removal:")
print("Train:", train.shape)
print("Valid:", valid.shape)
print("Test :", test.shape)



Dropping 37 columns due to >30% missing:
['MachineHoursCurrentMeter', 'UsageBand', 'fiSecondaryDesc', 'fiModelSeries', 'fiModelDescriptor', 'ProductSize', 'Drive_System', 'Forks', 'Pad_Type', 'Ride_Control', 'Stick', 'Transmission', 'Turbocharged', 'Blade_Extension', 'Blade_Width', 'Enclosure_Type', 'Engine_Horsepower', 'Pushblock', 'Ripper', 'Scarifier', 'Tip_Control', 'Tire_Size', 'Coupler', 'Coupler_System', 'Grouser_Tracks', 'Hydraulics_Flow', 'Track_Type', 'Undercarriage_Pad_Width', 'Stick_Length', 'Thumb', 'Pattern_Changer', 'Grouser_Type', 'Backhoe_Mounting', 'Blade_Type', 'Travel_Controls', 'Differential_Type', 'Steering_Controls']

Shapes after safe column removal:
Train: (401125, 16)
Valid: (11573, 15)
Test : (12457, 15)


In [148]:
# Only saleyear may matter
for df in [train, valid, test]:
    df['saledate'] = pd.to_datetime(df['saledate'])
    df['saleYear'] = df['saledate'].dt.year
    df.drop('saledate', axis=1, inplace=True)

In [149]:
from sklearn.preprocessing import LabelEncoder
import numpy as np

# Handle missing values
def handle_missing(df):
    for col in df.columns:
        if df[col].dtype == "object":
            df[col] = df[col].fillna("Missing")
        else:
            df[col] = df[col].fillna(df[col].median())
    return df

# Encode categorical columns using Label Encoding
def encode_categoricals(df, label_encoders=None, fit=True):
    if label_encoders is None:
        label_encoders = {}

    for col in df.columns:
        if df[col].dtype == 'object':
            if fit:
                le = LabelEncoder()
                df[col] = le.fit_transform(df[col])
                label_encoders[col] = le
            elif col in label_encoders:
                le = label_encoders[col]
                # Add unseen labels to encoder classes
                unseen_labels = set(df[col].unique()) - set(le.classes_)
                if unseen_labels:
                    le.classes_ = np.concatenate([le.classes_, list(unseen_labels)])
                df[col] = le.transform(df[col])
    return df, label_encoders

# Apply steps to all datasets
datasets = {'train': train, 'valid': valid, 'test': test}

# Handle missing values
for name, df in datasets.items():
    df = handle_missing(df)
    datasets[name] = df  # update reference

# Encode categorical columns
train, encoders = encode_categoricals(datasets['train'], fit=True)
valid, _ = encode_categoricals(datasets['valid'], label_encoders=encoders, fit=False)
test, _ = encode_categoricals(datasets['test'], label_encoders=encoders, fit=False)

# Save back to variables
datasets['train'] = train
datasets['valid'] = valid
datasets['test'] = test


In [150]:
train.info()
test.info()
valid.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 401125 entries, 0 to 401124
Data columns (total 16 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   SalesID             401125 non-null  int64  
 1   SalePrice           401125 non-null  int64  
 2   MachineID           401125 non-null  int64  
 3   ModelID             401125 non-null  int64  
 4   datasource          401125 non-null  int64  
 5   auctioneerID        401125 non-null  float64
 6   YearMade            401125 non-null  int64  
 7   fiModelDesc         401125 non-null  int64  
 8   fiBaseModel         401125 non-null  int64  
 9   fiProductClassDesc  401125 non-null  int64  
 10  state               401125 non-null  int64  
 11  ProductGroup        401125 non-null  int64  
 12  ProductGroupDesc    401125 non-null  int64  
 13  Enclosure           401125 non-null  int64  
 14  Hydraulics          401125 non-null  int64  
 15  saleYear            401125 non-nul

In [151]:
# Detecting outliers using Z-score capping
from scipy.stats import zscore
import numpy as np

def cap_outliers_zscore(df, columns, threshold=3.0):
    """
    Caps values in specified columns based on Z-score threshold.
    """
    for col in columns:
        if df[col].dtype in ['int64', 'float64']:
            mean = df[col].mean()
            std = df[col].std()
            upper = mean + threshold * std
            lower = mean - threshold * std
            # Cap values outside the threshold
            df[col] = np.where(df[col] > upper, upper, df[col])
            df[col] = np.where(df[col] < lower, lower, df[col])
    return df

# Exclude identifiers and target variable
exclude_cols = ['SalesID', 'MachineID', 'SalePrice']
numeric_cols = [
    col for col in train.columns
    if train[col].dtype in ['int64', 'float64'] and col not in exclude_cols
]

# Apply Z-score capping on train, valid, and test datasets
datasets['train'] = cap_outliers_zscore(datasets['train'], numeric_cols)
datasets['valid'] = cap_outliers_zscore(datasets['valid'], numeric_cols)
datasets['test'] = cap_outliers_zscore(datasets['test'], numeric_cols)


In [153]:
from google.colab import files
uploaded = files.upload()

valid_solution = pd.read_csv('ValidSolution.csv')
valid_solution=valid_solution.drop(columns=['Usage'])

Saving ValidSolution.csv to ValidSolution (6).csv


In [154]:
# Prepare feature and target variable
from sklearn.model_selection import train_test_split

# Separate features and target
X_train = datasets['train'].drop(columns=['SalePrice'])
y_train = datasets['train']['SalePrice']

X_valid = datasets['valid'].copy()
X_valid = X_valid.merge(valid_solution, on='SalesID', how='left')
y_valid = X_valid['SalePrice']
X_valid = X_valid.drop(columns=['SalePrice'])

In [155]:
# Linear Regression
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np

# Train
lr = LinearRegression()
lr.fit(X_train, y_train)

# Predict
y_pred_lr = lr.predict(X_valid)

# Evaluate
rmse_lr = np.sqrt(mean_squared_error(y_valid, y_pred_lr))
r2_lr = r2_score(y_valid, y_pred_lr)

print("Linear Regression:")
print(f"  RMSE: {rmse_lr:.2f}")
print(f"  R²  : {r2_lr:.4f}")


Linear Regression:
  RMSE: 24230.72
  R²  : 0.1452


In [156]:
# Random Forest Regression
from sklearn.ensemble import RandomForestRegressor

# Train
rf = RandomForestRegressor(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)

# Predict
y_pred_rf = rf.predict(X_valid)

# Evaluate
rmse_rf = np.sqrt(mean_squared_error(y_valid, y_pred_rf))
r2_rf = r2_score(y_valid, y_pred_rf)

print("Random Forest Regressor:")
print(f"  RMSE: {rmse_rf:.2f}")
print(f"  R²  : {r2_rf:.4f}")


Random Forest Regressor:
  RMSE: 10108.97
  R²  : 0.8512


In [157]:
# Gradient Boosting Regression
from sklearn.ensemble import GradientBoostingRegressor

# Train
gb = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, random_state=42)
gb.fit(X_train, y_train)

# Predict
y_pred_gb = gb.predict(X_valid)

# Evaluate
rmse_gb = np.sqrt(mean_squared_error(y_valid, y_pred_gb))
r2_gb = r2_score(y_valid, y_pred_gb)

print("Gradient Boosting Regressor:")
print(f"  RMSE: {rmse_gb:.2f}")
print(f"  R²  : {r2_gb:.4f}")


Gradient Boosting Regressor:
  RMSE: 15092.82
  R²  : 0.6684


In [158]:
# Predict on test set using random forest model as it has greatest R2
y_test_pred = rf.predict(datasets['test'])


submission_df = pd.DataFrame({
    'SalesID': datasets['test']['SalesID'],
    'SalePrice': y_test_pred
})

submission_df.to_csv('test_predictions_rf.csv', index=False)

print(submission_df.head())
files.download('test_predictions_rf.csv')

   SalesID  SalePrice
0  1227829    12937.5
1  1227844    25570.0
2  1227847    51765.0
3  1227848   102010.0
4  1227863    25470.0


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>